In [ ]:
import os

# Path to the dataset directory
data_dir = "./New Plant Diseases Dataset(Augmented)"

# List the contents of the dataset directory
print("Contents of data_dir:", os.listdir(data_dir))


In [ ]:
import os                       # for working with files
import numpy as np              # for numerical computationss
import pandas as pd             # for working with dataframes
import torch                    # Pytorch module 
import matplotlib.pyplot as plt # for plotting informations on graph and images using tensors
import torch.nn as nn           # for creating  neural networks
from torch.utils.data import DataLoader # for dataloaders 
from PIL import Image           # for checking images
import torch.nn.functional as F # for functions for calculating loss
import torchvision.transforms as transforms   # for transforming images into tensors 
from torchvision.utils import make_grid       # for data checking
from torchvision.datasets import ImageFolder  # for working with classes and images
from torchsummary import summary              # for getting the summary of our model

%matplotlib inline

In [ ]:
import os

# Path to the dataset directory (relative path)
data_dir = "New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)"
train_dir = os.path.join(data_dir, "train")
valid_dir = os.path.join(data_dir, "valid")

# List contents of the directories
print("Contents of data_dir:", os.listdir(data_dir))
print("Contents of train_dir:", os.listdir(train_dir))
print("Contents of valid_dir:", os.listdir(valid_dir))

train_dir = data_dir + "/train"
valid_dir = data_dir + "/valid"
diseases = os.listdir(train_dir)
print("Diseases:", diseases)


In [ ]:
plants = []
NumberOfDiseases = 0
for plant in diseases:
    if plant.split('___')[0] not in plants:
        plants.append(plant.split('___')[0])
    if plant.split('___')[1] != 'healthy':
        NumberOfDiseases += 1
print(plants)
# Save plants array to a file
with open('plants.txt', 'w') as file:
    for plant in plants:
        file.write(plant + '\n')


In [ ]:
# Number of images for each disease

# Save diseases list to a new text file
with open('diseases.txt', 'w') as file:
    for disease in diseases:
        file.write(disease + '\n')
nums = {}
for disease in diseases:
    nums[disease] = len(os.listdir(train_dir + '/' + disease))
    
# converting the nums dictionary to pandas dataframe passing index as plant name and number of images as column

img_per_class = pd.DataFrame(nums.values(), index=nums.keys(), columns=["no. of images"])
img_per_class

In [ ]:
n_train = 0
for value in nums.values():
    n_train += value
print(f"There are {n_train} images for training")

In [ ]:
# datasets for validation and training
train = ImageFolder(train_dir, transform=transforms.ToTensor())
valid = ImageFolder(valid_dir, transform=transforms.ToTensor()) 

In [ ]:
import torch

# For selecting the default device (GPU if available, else CPU)
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():  # Add parentheses to properly call the function
        return torch.device("cuda")
    else:
        return torch.device("cpu")

# For moving data to the chosen device
def to_device(data, device):
    """Move tensor(s) to the chosen device (CPU or GPU)"""
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

# Wrapper for DataLoader to automatically move batches to the chosen device
class DeviceDataLoader:
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)
        
    def __len__(self):
        """Number of batches"""
        return len(self.dl)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# For calculating the accuracy
def accuracy(outputs, labels):
    """
    Calculate accuracy: Compares predictions with actual labels.
    Args:
        outputs: Model predictions (logits or probabilities).
        labels: Actual labels.
    Returns:
        Accuracy as a PyTorch tensor.
    """
    _, preds = torch.max(outputs, dim=1)  # Get predicted class indices
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))  # Compute accuracy


# Base class for image classification models
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        """
        Training step: Computes loss for a batch.
        Args:
            batch: A tuple containing images and labels.
        Returns:
            Loss value.
        """
        images, labels = batch
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate cross-entropy loss
        return loss
    
    def validation_step(self, batch):
        """
        Validation step: Computes loss and accuracy for a batch.
        Args:
            batch: A tuple containing images and labels.
        Returns:
            A dictionary with validation loss and accuracy.
        """
        images, labels = batch
        out = self(images)                   # Generate predictions
        loss = F.cross_entropy(out, labels)  # Calculate loss
        acc = accuracy(out, labels)          # Calculate accuracy
        return {"val_loss": loss.detach(), "val_accuracy": acc}
    
    def validation_epoch_end(self, outputs):
        """
        Combine outputs of validation steps across an epoch.
        Args:
            outputs: List of dictionaries from validation steps.
        Returns:
            A dictionary with average validation loss and accuracy.
        """
        batch_losses = [x["val_loss"] for x in outputs]
        batch_accuracy = [x["val_accuracy"] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()  # Average loss
        epoch_accuracy = torch.stack(batch_accuracy).mean()  # Average accuracy
        return {"val_loss": epoch_loss, "val_accuracy": epoch_accuracy}
    
    def epoch_end(self, epoch, result):
        """
        Print epoch metrics.
        Args:
            epoch: Current epoch number.
            result: Dictionary containing epoch metrics (loss, accuracy, etc.).
        """
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result.get('train_loss', 0), result['val_loss'], result['val_accuracy']))


In [ ]:
import torch
import torch.nn as nn

# Convolution block with BatchNormalization and optional MaxPooling
def ConvBlock(in_channels, out_channels, pool=False):
    """
    Creates a convolutional block with BatchNorm and ReLU activation.
    Args:
        in_channels: Number of input channels.
        out_channels: Number of output channels.
        pool: Whether to apply MaxPooling.
    Returns:
        A sequential block with Conv2D, BatchNorm, ReLU, and optional MaxPooling.
    """
    layers = [
        nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    ]
    if pool:
        layers.append(nn.MaxPool2d(4))  # Pooling with kernel size of 4
    return nn.Sequential(*layers)


# ResNet9 architecture
class ResNet9(ImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        """
        Initializes the ResNet9 model.
        Args:
            in_channels: Number of input channels (e.g., 3 for RGB images).
            num_classes: Number of output classes (e.g., number of diseases).
        """
        super().__init__()
        
        # Initial convolutional layers
        self.conv1 = ConvBlock(in_channels, 64)
        self.conv2 = ConvBlock(64, 128, pool=True)  # Downsample with pooling
        self.res1 = nn.Sequential(
            ConvBlock(128, 128),
            ConvBlock(128, 128)
        )  # First residual block
        
        # Deeper convolutional layers
        self.conv3 = ConvBlock(128, 256, pool=True)
        self.conv4 = ConvBlock(256, 512, pool=True)
        self.res2 = nn.Sequential(
            ConvBlock(512, 512),
            ConvBlock(512, 512)
        )  # Second residual block
        
        # Classifier head
        self.classifier = nn.Sequential(
            nn.MaxPool2d(4),  # Global average pooling
            nn.Flatten(),     # Flatten for linear layer
            nn.Linear(512, num_classes)
        )
        
    def forward(self, xb):
        """
        Forward pass through the network.
        Args:
            xb: Input batch of images.
        Returns:
            Output logits for each class.
        """
        # Initial convolutional layers
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out  # Residual connection
        
        # Deeper convolutional layers
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out  # Residual connection
        
        # Classifier head
        out = self.classifier(out)
        return out


In [ ]:

test = ImageFolder(valid_dir, transform=transforms.ToTensor())

In [ ]:
test_images = sorted(os.listdir(valid_dir + '/Apple___Apple_scab')) # since images in test folder are in alphabetical order
test_images

In [ ]:
def predict_image(img, model):
    """Converts image to array and return the predicted class
        with highest probability"""
    # Convert to a batch of 1
    xb = to_device(img.unsqueeze(0), device)
    # Get predictions from model
    yb = model(xb)
    # Pick index with highest probability
    _, preds  = torch.max(yb, dim=1)
    # Retrieve the class label

    return train.classes[preds[0].item()]

In [ ]:
import torch
from torchvision import transforms

# Load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet9(3, len(train.classes))  # Define the same model architecture
model.load_state_dict(torch.load('model.pth', map_location=device))  # Load the model weights onto the device (CPU or GPU)
model.to(device)  # Move the model to the appropriate device (GPU or CPU)
model.eval()  # Set the model to evaluation mode



# predicting first image
img, label = test[0]
plt.imshow(img.permute(1, 2, 0))

In [ ]:
# getting all predictions (actual label vs predicted)
for i, (img, label) in enumerate(test):
    print('Label:', test_images[i], ', Predicted:', predict_image(img, model))